In [45]:
#pip install requests

Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests
import json
import json
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from IPython.display import clear_output
from matplotlib.pyplot import figure
import ipywidgets as widgets
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

In [37]:
def graph_refresh():
    with open('data_file.json') as json_file:
        prices = json.load(json_file)

    ge_prices = pd.DataFrame.from_dict(prices['data'])
    ge_prices = ge_prices.transpose()
    ge_prices.reset_index(level=0, inplace=True)
    ge_prices = ge_prices.rename(columns={"index": "id"})

    with open('mapping_file.json') as json_file:
        mappings = json.load(json_file)

    id_mappings = pd.DataFrame.from_dict(mappings)

    ge_prices['id']=ge_prices['id'].astype(int)
    id_mappings['id']=id_mappings['id'].astype(int)

    ge_sot = pd.merge(ge_prices, id_mappings, how="left", on='id')
    
    ge_sot_highalch_profit = ge_sot[ge_sot["highalch"]-ge_sot["high"]>0]
    ge_sot_highalch_profit = ge_sot_highalch_profit[ge_sot_highalch_profit["high"]> ge_sot_highalch_profit["low"]]
    ge_sot_highalch_profit["profit"] = ge_sot_highalch_profit["highalch"]-ge_sot_highalch_profit["high"]
    ge_sot_highalch_profit = ge_sot_highalch_profit.sort_values(by=['profit'], ascending=False)
    
    
    highalch_top_5 = ge_sot_highalch_profit[:5]
    
    clear_output(wait=True)
    
    x = datetime.datetime.now()
    prices = highalch_top_5["name"],highalch_top_5["high"]
    
    plt.rcParams['figure.figsize'] = [10, 5]
    plt.bar(range(len(highalch_top_5["profit"])), highalch_top_5["profit"], align='center')
    plt.xticks(range(len(highalch_top_5["name"])), highalch_top_5["name"], size='small')
    
    for i, v in enumerate(highalch_top_5["high"]):
        plt.text(i-0.2, highalch_top_5["profit"].iloc[i] +15, str(v))
    
    plt.xlabel('Item name')
    plt.ylabel('Profit')
    plt.title(str(x))
    plt.autoscale()
    plt.show()
    
    return highalch_top_5

In [45]:
from time import time, sleep
def runjob():

    prices = r'https://prices.runescape.wiki/api/v1/osrs/latest'

    mappings = r'https://prices.runescape.wiki/api/v1/osrs/mapping'

    headers = {
        'User-Agent': 'Khealim user agent',
        'From': 'tamaskatona94@gmail.com'  
    }
    
    response = requests.get(prices, headers=headers)
    data = response.json()
    with open("data_file.json", "w") as write_file:
        json.dump(data, write_file)
    #response = requests.get(mappings, headers=headers)
    #data = response.json()
    #with open("mapping_file.json", "w") as write_file:
    #    json.dump(data, write_file)
        
    prices = graph_refresh()

In [43]:
def on_button_clicked(b):
    with out:
        clear_output()
        runjob()

In [46]:
from IPython.display import display, clear_output
import ipywidgets as widgets
import random

b = widgets.Button(
    description='Refresh',
    disabled=False,
    button_style='info',
    tooltip='Click me',
    icon='check'
)
display(b)

out = widgets.Output()
display(out)

b.on_click(on_button_clicked)

Button(button_style='info', description='Refresh', icon='check', style=ButtonStyle(), tooltip='Click me')

Output()